In [1]:
#Python 2 compatibility stuff
from __future__ import print_function
from __future__ import division
from builtins import str
from builtins import range
from six.moves import configparser
import six
if six.PY2:
    ConfigParser = configparser.SafeConfigParser
else:
    ConfigParser = configparser.ConfigParser

In [2]:
import numpy as np
from szar.derivatives import Derivs_Clustering, make_fisher

In [3]:
EXPNAME = 'S4-1.0-CDT'
GRIDNAME = 'grid-owl2'
VERSION = '0.6'
saveid = EXPNAME + "_" + GRIDNAME + "_v" + VERSION + '_'
DIR = "../datatest/"
TIMESTAMP = "_2018-11-05-10-26-09-EST"
DERIVSFILE = saveid + "fish_derivs" + TIMESTAMP + '.npy'
FACTORFILE = saveid + "fish_factor" + TIMESTAMP + '.npy'
config = ConfigParser()
config.optionxform=str
INIFILE = "../input/pipeline.ini"
config.read(INIFILE)

['../input/pipeline.ini']

In [4]:
fisher_derivs = np.load(DIR + DERIVSFILE)
fisher_facs = np.load(DIR + FACTORFILE)

In [5]:
deriv = Derivs_Clustering(INIFILE, EXPNAME, GRIDNAME, VERSION)
deriv.instantiate_params()
params = deriv.params
fisher_params = deriv.fisher_params
fisher_params

{'omch2': 0.1194,
 'ombh2': 0.022,
 'H0': 67.0,
 'ns': 0.96,
 'As': 2.2e-09,
 'mnu': 0.06,
 'w0': -1.0,
 'wa': 0.0,
 'alpha_ym': 1.79,
 'b_ym': 0.8,
 'beta_ym': 0.0,
 'gamma_ym': 0.0,
 'Ysig': 0.127,
 'gammaYsig': 0.0,
 'betaYsig': 0.0,
 'b_wl': 1.0}

In [6]:
params_oom = np.array([0.1, 0.1, 10, 1, 1e-9, 0.01, 1, 1, 1, 1, 1, 1, 0.1, 1, 1, 1])
fisher_mat = make_fisher(fisher_derivs, fisher_facs)
params_mat = np.outer(params_oom,params_oom)
fisher_renorm = params_mat * fisher_mat

In [7]:
fisher = fisher_renorm[:-1]
fisher = fisher.T[:-1].T

In [8]:
param_index = {key:index for index,key in enumerate(fisher_params.keys())}

In [9]:
param_index

{'omch2': 0,
 'ombh2': 1,
 'H0': 2,
 'ns': 3,
 'As': 4,
 'mnu': 5,
 'w0': 6,
 'wa': 7,
 'alpha_ym': 8,
 'b_ym': 9,
 'beta_ym': 10,
 'gamma_ym': 11,
 'Ysig': 12,
 'gammaYsig': 13,
 'betaYsig': 14,
 'b_wl': 15}

In [16]:
covmat = np.linalg.inv(fisher)
#u,s,v = np.linalg.svd(fisher)
#covmat = np.matmul(v.transpose(), np.matmul(np.diag(s**-1), u.transpose()))

In [11]:
def plot_cov_ellipse(cov, xval, yval, ax=None):
    if ax is None:
        ax = plt.gca()
    xx = np.array(np.arange(360) / 180. * np.pi)
    circl = np.array([np.cos(xx),np.sin(xx)])
    Lmat = np.linalg.cholesky(cov)
    ansout = np.dot(1.52*Lmat,circl)
    ax.plot(ansout[0,:]+xval, ansout[1,:]+yval)

In [12]:
def make_ellipse(param1, param2):
    sigma_h0omch2 = covmat[param_index[param1], param_index[param2]]
    sigma_omch2h0 = covmat[param_index[param2], param_index[param1]]
    sigma_h0_sqr = covmat[param_index[param1], param_index[param1]]
    sigma_omch2_sqr = covmat[param_index[param2], param_index[param2]]
    cov_h0_omch2 = np.array([[sigma_h0_sqr, sigma_h0omch2],[sigma_omch2h0, sigma_omch2_sqr]])
    plot_cov_ellipse(cov_h0_omch2, params[param1], params[param2], ax=None)
    plt.show()

In [13]:
make_ellipse('gamma_ym', 'H0')

NameError: name 'plt' is not defined

In [14]:
params

{'omch2': 0.1194,
 'ombh2': 0.022,
 'H0': 67.0,
 'ns': 0.96,
 'As': 2.2e-09,
 'mnu': 0.06,
 'w0': -1.0,
 'wa': 0.0,
 'alpha_ym': 1.79,
 'b_ym': 0.8,
 'beta_ym': 0.0,
 'gamma_ym': 0.0,
 'Ysig': 0.127,
 'gammaYsig': 0.0,
 'betaYsig': 0.0,
 'b_wl': 1.0,
 'tau': 0.06,
 'Y_star': 2.42e-10}

In [15]:
idtest = np.matmul(fisher,covmat)
idtest

array([[ 1.00000000e+00, -1.39978289e-14, -1.20186449e-12,
        -1.27056073e-13,  1.31894525e-10,  6.70962672e-10,
         1.98040856e-11,  5.33275978e-11, -3.58602029e-11,
        -1.82343570e-10,  1.48542251e-09,  2.07621955e-09,
         4.77957553e-09, -4.81968355e-08, -3.58565055e-08],
       [ 3.11424874e-14,  1.00000000e+00,  1.01468766e-12,
         2.63925860e-14, -2.04074328e-11, -8.79917437e-11,
        -3.11791893e-12, -9.69373876e-12,  1.41056952e-10,
         3.62628088e-11, -8.74033138e-11, -3.95297731e-10,
        -1.07677278e-09,  1.05773464e-08,  4.71489713e-09],
       [-4.30159595e-13, -1.43605956e-13,  1.00000000e+00,
         5.20959554e-14, -1.39759618e-12, -9.26690921e-11,
         2.94535591e-13,  3.03865898e-12,  1.81446761e-10,
        -3.23557194e-12, -2.41293313e-10,  8.48904987e-11,
        -7.25196058e-11,  4.33800133e-10, -7.27871546e-09],
       [-3.87538791e-13, -2.74590280e-13, -2.12420972e-12,
         1.00000000e+00,  3.20218144e-11,  2.76876747